## Import and Data Path Assignment

To begin our analysis, we first need to import the necessary libraries and assign the path to our data files. This step ensures that we have access to the tools and resources required for the subsequent tasks.

Let's get started by executing the following code:

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

So, we have few options to open microscopy datafiles:

- skimage.io.imread
- skimage.io.imread_collection
- tifffile.TiffFile
- tifffile.TiffSequence
- bioformats.ImageReader

Imagej hyperstack are organized as **TZCYXS**.

Holoview can also be used. Check availability of reading from disk a la memmap.

Bioformats claims the following standard: separate tiff for each channel and for each time point.
Thus a ome.tif would contain a single plane or a zstack. 

What about tiles? Bioformats has some 6D, 7D and 8D working aroud 5D. But how is exactly defined 5D?

## Skimage and Tifffile

In [ ]:
t1 = skimage.io.imread(img_tile, plugin='tifffile')
t2 = skimage.io.imread(img_void_tile, plugin='tifffile')
t1.shape, t2.shape

In [ ]:
tf1 = tifffile.imread(img_tile)
tf2 = tifffile.imread(img_void_tile)
tf1.shape, tf2.shape

Only for tiff data files provides:

- sequence
- OME metadata
- memmap
- zarr

In [ ]:
fp1glob = '/home/dati/liaisan/data/260816/FLAT/cyan_em10/cyan_em10_p*.tif'
tifs = tifffile.TiffSequence(fp1glob)
d = tifs.asarray()
print(d.shape) 
print(tifs.shape)

In [ ]:
with tifffile.TiffFile(img_tile) as tif:
    tag = tif.pages[0].tags['ImageDescription']
    
tag.value

## nima_io.read

In [ ]:
ir.ensure_vm()

In [ ]:
md1, wr1 = ir.read(str(img_tile))
md2, wr2 = ir.read(str(img_void_tile))

In [ ]:
md1, md2

In [ ]:
f = ir.stitch(md2, wr2, c=2, t=2)
skimage.io.imshow(f)

In [ ]:
#import javabridge
#import bioformats
#javabridge.start_vm(class_path=bioformats.JARS)

## performance
### showinf

In [ ]:
%timeit ir.read_inf(lif), ir.read_inf(img), ir.read_inf(img_tile)
%memit ir.read_inf(lif), ir.read_inf(img), ir.read_inf(img_tile)

### bioformats (std)

In [ ]:
%timeit ir.read_bf(lif), ir.read_bf(img), ir.read_bf(img_tile)
%memit ir.read_bf(lif), ir.read_bf(img), ir.read_bf(img_tile)

### javabridge

In [ ]:
# OME-TIFF only
%timeit ir.read_jb(img), ir.read_jb(img_tile)
%memit ir.read_jb(img), ir.read_jb(img_tile)

### read

In [ ]:
%timeit ir.read(lif), ir.read(img), ir.read(img_tile)
%memit ir.read(lif), ir.read(img), ir.read(img_tile)

## ImageIO

Started comparing the available options to read microscopy file.

- mosaic

In [ ]:
from imageio.v3 import imread

%timeit i = imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [ ]:
from aicsimageio import AICSImage

%timeit i = AICSImage(img_void_tile)
i = AICSImage(img_void_tile, reconstruct_mosaic=True)
i_lif = AICSImage(lif)

In [ ]:
i_lif.scenes

In [ ]:
i.get_dask_stack()

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [ ]:
from dask_image.imread import imread

imread(img_void_tile)

In [ ]:
imread(lif)

Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to intall.

**pyimagej** need conda?

## bioio-bioformats

In [ ]:
import bioio_bioformats

%timeit bioio_bioformats.Reader(img_void_tile)

In [ ]:
i = bioio_bioformats.Reader(img_void_tile)
i.data.shape

In [ ]:
i.get_xarray_dask_stack()

In [ ]:
stk = i.get_dask_stack()

In [ ]:
stk.A

## bfio

In [ ]:
import bfio

%timeit bfio.BioReader(img_void_tile)

In [ ]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

In [ ]:
rdr.metadata

In [ ]:
rdr.ps_x

## PIMS

In [ ]:
import pims

%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

## PyOMETiff

In [ ]:
import pyometiff

%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

In [ ]:
%timeit r = rdr.read()
res = rdr.read()

In [ ]:
res[2]

In [ ]:
pyometiff.OMETIFFReader._get_metadata_template()

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping


- pycromanager
- microscope
- python-microscopy

In [ ]:
import imgread

In [ ]:
from imgread import read

In [ ]:
read.read_pims(img_void_tile)

In [ ]:
read.read_jb(img_void_tile)